Load packages and set up working directory.

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.io
import os
import time
import io

In [2]:
os.chdir('/Project/lstm_prediction/LSTM1/')

Read files.

The function to standardize a matrix by columns.

In [3]:
def standardize(a):
    nrow = a.shape[0]
    mu = np.mean(a, axis=0, keepdims=True)
    sig = np.std(a, axis=0, keepdims=True)
    a = (a - np.repeat(mu, repeats=nrow, axis=0))/np.repeat(sig, repeats=nrow, axis=0)
    return a

In [4]:
ADHD = scipy.io.loadmat('/Project/lstm_prediction/ADHD_roimean.mat')
adhd_roimean = ADHD['roimean']
adhd_roimean = [standardize(m[0]) for m in adhd_roimean]
adhd_response = standardize(np.squeeze(ADHD['ADHD_score'].T))
#adhd_response = np.squeeze(ADHD['ADHD_score'].T)

In [6]:
ANT = scipy.io.loadmat('/Project/lstm_prediction/ANT_roimean.mat')
ANT_rest = ANT['roimean_res']
ANT_task = [m for m in ANT['roimean_task'][0]]
ANT_response = ANT['ANTbehav']
tmp_mean = np.repeat(np.array([np.mean(ANT['ANTbehav'], axis=0)]), axis=0, repeats=ANT['ANTbehav'].shape[0])
tmp_std = np.repeat(np.array([np.std(ANT['ANTbehav'], axis=0)]), axis=0, repeats=ANT['ANTbehav'].shape[0])
ANT_response = (ANT_response - tmp_mean) / tmp_std
ANT_response = ANT_response[:, 0]

In [7]:
regions = np.genfromtxt('/Project/lstm_prediction/labels_ADHD.csv')

In [8]:
gdir = '/Project/lstm_prediction/gradCPTroimean/'
gradCPT = scipy.io.loadmat('/Project/lstm_prediction/gradCPT_data_share.mat')
gorder = np.genfromtxt('/Project/lstm_prediction/subject_order.txt', dtype='int')
gradCPT_roimean = []
for i in np.arange(len(gorder)):
    fname = gdir + str(gorder[i]) + '_FullRest_matrix_bis_matrix_roimean.txt'
    temp = np.genfromtxt(fname, delimiter='\t', skip_header=True, )
    temp = temp[:, 1:269]
    temp = temp[:,regions!=21]
    gradCPT_roimean.append(temp)
gradCPT_response = np.squeeze(standardize(gradCPT['dprime']))
#gradCPT_response = np.squeeze(gradCPT['dprime'].T)

Basic parameter setting.<br>
num\_steps: number of steps to trace<br>
batch_size: number of sampels for each batch<br>
state_size: number of states in GRU<br>
num_layers: number of layers of multilayer GRU<br>
learning_rate: step size in gradient descent<br>

In [9]:
num_steps = 100
batch_size = 100
state_size = 256
num_layers = 1
learning_rate = [1e-5, 1e-5]
input_x_dim = 268
w_slice = 10

Function to generate batch with slice. If the slice size equals the step length, there will be no overlap between sampled time windows.

In [10]:
def gen_batch(raw_data_x, raw_data_y, num_steps, batch_size, w_slice = 1):
    N = len(raw_data_x)
    N_k = np.array([int((raw_data_x[i].shape[0] - num_steps) / w_slice)+1 for i in np.arange(N)])
    epochs = np.concatenate([np.arange(0, n_k*w_slice, w_slice) for n_k in N_k])
    Ns = np.concatenate([np.ones(y)*x for x, y in zip(np.arange(N), N_k)])
    samples = np.array([[x, y] for x, y in zip(Ns, epochs)])
    np.random.shuffle(samples)
    epoch_size = samples.shape[0] // batch_size
    samples = np.concatenate([samples, samples[0:(batch_size - samples.shape[0] + batch_size * epoch_size),]])
    epoch_size = samples.shape[0] //batch_size
    for i in range(epoch_size):
        x = np.array([raw_data_x[int(samples[k, 0])]\
             [int(samples[k, 1]):(int(samples[k, 1]) + num_steps),] \
             for k in np.arange(i * batch_size, (i + 1) * batch_size)])
        y = raw_data_y[np.int32(samples[np.arange(i * batch_size, (i + 1) * batch_size),0])]
        z = np.int32(samples[np.arange(i * batch_size, (i + 1) * batch_size),0])
        yield (x, y, z)
    

Function to generate epoch.

In [11]:
def gen_epochs(num_epochs, raw_data_x, raw_data_y, num_steps, batch_size, w_slice=1):
    for i in range(num_epochs):
        yield gen_batch(raw_data_x, raw_data_y, num_steps, batch_size)

Function to reset graph.<br>
ref: https://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html

In [12]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

Average gradients for multi-tower design:
https://github.com/petewarden/tensorflow_makefile/blob/master/tensorflow/models/image/cifar10/cifar10_multi_gpu_train.py

In [13]:
def average_gradients(tower_grads):
    average_grads = []
    for grad_and_vars in zip(*tower_grads):
    # Note that each grad_and_vars looks like the following:
    #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
        grads = []
        for g, _ in grad_and_vars:
      # Add 0 dimension to the gradients to represent the tower.
            expanded_g = tf.expand_dims(g, 0)

      # Append on a 'tower' dimension which we will average over below.
            grads.append(expanded_g)

    # Average over the 'tower' dimension.
        grad = tf.concat(axis=0, values=grads)
        grad = tf.reduce_mean(grad, 0)

    # Keep in mind that the Variables are redundant because they are shared
    # across towers. So .. we will just return the first tower's pointer to
    # the Variable.
        v = grad_and_vars[0][1]
        grad_and_var = (grad, v)
        average_grads.append(grad_and_var)
    return average_grads

Function to build computational graph.

In [1]:
def build_graph(
        state_size, 
        batch_size, 
        num_steps, 
        num_layers, 
        learning_rate,
        input_x_dim,
        hidden_states=256,
        devices = ['/gpu:0'],
        reg_scale=[0.001, 0.005],
        keep_prob=[1, 0.5]):
    reset_graph()
    with tf.device('/cpu:0'):
        x = tf.placeholder(tf.float32, [batch_size, num_steps, input_x_dim], name = 'fMRI')
        y = tf.placeholder(tf.float32, [batch_size], name = 'Response')
        cell = tf.contrib.rnn.GRUCell(state_size, activation=tf.nn.relu6)
        cell_to_stack = tf.contrib.rnn.GRUCell(state_size, activation=tf.nn.relu6)
        cell = tf.contrib.rnn.MultiRNNCell([cell] + [cell_to_stack] * (num_layers - 1), state_is_tuple = True)
        cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=keep_prob[0])
        global_step = tf.get_variable('global_step', shape = [], trainable=False, initializer = tf.constant_initializer(0))
        init_state = cell.zero_state(batch_size/len(devices), tf.float32)
        x_splits = tf.split(x, num_or_size_splits=len(devices))
        y_splits = tf.split(y, num_or_size_splits=len(devices))
        lr1 = tf.train.exponential_decay(learning_rate[0],
                                                 global_step = global_step,
                                                 decay_steps = 30,
                                                 decay_rate = 0.96,
                                                 staircase=True)
        lr2 = tf.train.exponential_decay(learning_rate[1],
                                                 global_step = global_step,
                                                 decay_steps = 30,
                                                 decay_rate = 0.96,
                                                 staircase=True)
        opt1 = tf.train.AdamOptimizer(lr1)
        opt2 = tf.train.AdamOptimizer(lr2)
        with tf.variable_scope('Final'):
            with tf.variable_scope('Dense1'):
                W1 = tf.get_variable('W', [state_size,hidden_states])
                b1 = tf.get_variable('bias', [hidden_states], initializer = tf.constant_initializer(0.0))
            with tf.variable_scope('Dense2'):
                W2 = tf.get_variable('W', [hidden_states, hidden_states])
                b2 = tf.get_variable('bias', [hidden_states], initializer = tf.constant_initializer(0.0))                
            with tf.variable_scope('output'):
                Wo = tf.get_variable('W', [hidden_states, 1])
                bo = tf.get_variable('bias', [1], initializer = tf.constant_initializer(0.0))
        total_loss = []
        outputs = []
        grads1 = []
        grads2 = []
        l1_regularizer1 = tf.contrib.layers.l1_regularizer(scale=reg_scale[0], scope=None)
        l1_regularizer2 = tf.contrib.layers.l1_regularizer(scale=reg_scale[1], scope=None)
        for i in range(len(devices)):
            d = devices[i]
            with tf.device(d):   
                rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, x_splits[i], initial_state = init_state)
                
                rnn_output = rnn_outputs[:,-1,:]                
                dense1 = tf.nn.relu(tf.matmul(rnn_output, W1) + b1)
                dense1 = tf.nn.dropout(dense1, keep_prob = keep_prob[1])    
                dense2 = tf.nn.relu(tf.matmul(dense1, W2) + b2)
                dense2 = tf.nn.dropout(dense2, keep_prob = keep_prob[1])    
                output = tf.squeeze(tf.matmul(dense2, Wo) + bo)
                outputs.append(output)
                
                w_output =[tf_var for tf_var in tf.trainable_variables() if not ("rnn" in tf_var.name or "bias" in tf_var.name)]
                regularization_penalty_output = tf.contrib.layers.apply_regularization(l1_regularizer1, w_output)
                w_rnn =[tf_var for tf_var in tf.trainable_variables() if not ("Final" in tf_var.name or "bias" in tf_var.name)]
                regularization_penalty_rnn = tf.contrib.layers.apply_regularization(l1_regularizer2, w_rnn)
                
                loss = tf.reduce_mean(tf.abs(output - y_splits[i])) + regularization_penalty_output + regularization_penalty_rnn
                grad1 = opt1.compute_gradients(loss, var_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "rnn"))
                grad2 = opt2.compute_gradients(loss, var_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "Final"))
                grads1.append(grad1)
                grads2.append(grad2)
                total_loss.append(loss)  
        grads1 = average_gradients(grads1)
        grads2 = average_gradients(grads2)
        outputs = tf.concat(outputs, axis=0)
        total_loss = tf.reduce_mean(total_loss)
        
        train_step = tf.group(opt1.apply_gradients(grads1, global_step = global_step), 
                              opt2.apply_gradients(grads2, global_step = global_step), 
                              tf.assign_add(global_step, 1))
            
    
    return dict(
        x = x,
        y = y,
        prediction = outputs,
        init_state = init_state,
        final_state = final_state,
        total_loss = total_loss,
        train_step = train_step,
    )

In [18]:
def build_graph_simple(
        state_size, 
        batch_size, 
        num_steps, 
        num_layers, 
        learning_rate,
        input_x_dim,
        reg_scale=[0.001, 0.005],
        keep_prob=[1, 0.5]):
    reset_graph()
    with tf.device('/gpu:0'):
        x = tf.placeholder(tf.float32, [batch_size, num_steps, input_x_dim], name = 'fMRI')
        y = tf.placeholder(tf.float32, [batch_size], name = 'Response')
        
        global_step = tf.get_variable('global_step', shape = [], trainable=False, initializer = tf.constant_initializer(0))
        cell = tf.contrib.rnn.GRUCell(state_size, activation = tf.nn.relu6)
        cell_to_stack = tf.contrib.rnn.GRUCell(state_size, activation = tf.nn.relu6)
        cell = tf.contrib.rnn.MultiRNNCell([cell] + [cell_to_stack] * (num_layers - 1), state_is_tuple = True)
        cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=keep_prob[0])
            
        init_state = cell.zero_state(batch_size, tf.float32)
        rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, x, initial_state = init_state)
        
        rnn_output = rnn_outputs[:,-1,:]
        
        with tf.variable_scope('Final', reuse = True):
        
            with tf.variable_scope('output'):
                Wo = tf.get_variable('W', [state_size, 1])
                bo = tf.get_variable('bias', [1], initializer = tf.constant_initializer(0.0))
            
    
        dense3 = tf.nn.dropout(rnn_output, keep_prob = keep_prob[1])    
        output = tf.squeeze(tf.matmul(dense3, Wo) + bo)
        total_loss = tf.reduce_mean(tf.squared_difference(output, y))
        l1_regularizer1 = tf.contrib.layers.l1_regularizer(scale=reg_scale[0], scope=None)
        l1_regularizer2 = tf.contrib.layers.l1_regularizer(scale=reg_scale[1], scope=None)
        w_output =[tf_var for tf_var in tf.trainable_variables() if not ("rnn" in tf_var.name or "bias" in tf_var.name)]
        regularization_penalty_output = tf.contrib.layers.apply_regularization(l1_regularizer1, w_output)
        w_rnn =[tf_var for tf_var in tf.trainable_variables() if not ("Final" in tf_var.name or "bias" in tf_var.name)]
        regularization_penalty_rnn = tf.contrib.layers.apply_regularization(l1_regularizer2, w_rnn)    
        regularized_loss = total_loss + regularization_penalty_output + regularization_penalty_rnn
        vars1 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "rnn")
        vars2 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "Final")
                
        lr1 = tf.train.exponential_decay(learning_rate[0],
                                         global_step = global_step,
                                         decay_steps = 100,
                                         decay_rate = 0.96,
                                         staircase=True)
        lr2 = tf.train.exponential_decay(learning_rate[1],
                                         global_step = global_step,
                                         decay_steps = 100,
                                         decay_rate = 0.96,
                                         staircase=True)

        train_step1 = tf.train.AdamOptimizer(lr1).minimize(regularized_loss, var_list = vars1)
        train_step2 = tf.train.AdamOptimizer(lr2).minimize(regularized_loss, var_list = vars2)
        train_step = tf.group(train_step1, train_step2, tf.assign_add(global_step, 1))
    
    return dict(
        x = x,
        y = y,
        prediction = output,
        init_state = init_state,
        final_state = final_state,
        total_loss = total_loss,
        train_step = train_step,
        var = [vars1, vars2]
    )

In [16]:
def train_network(g, num_epochs, num_steps, 
                  batch_size, raw_data_x, raw_data_y, 
                  valid_data_x = None, valid_data_y = None, w_slice = 1,
                  save = False, sess = None):
    valid = False
    if (valid_data_x is not None) and (valid_data_y is not None):
        valid = True
    if sess is None:
        sess = tf.Session()
        sess.run(tf.global_variables_initializer())
    training_losses = []
    for idx, epoch in enumerate(gen_epochs(num_epochs, raw_data_x, 
                                           raw_data_y, num_steps, batch_size, w_slice)):
        training_loss = 0
        steps = 0
        flag = 0
        for X, Y, Z in epoch:
            steps = steps + 1
            feed_dict = {g['x']: X, g['y']: Y}
            training_loss_, _ = sess.run([g['total_loss'], g['train_step']],feed_dict)
            if flag == 0:
                flag = 1
                if valid:
                    valid_loss = 0
                    valid_r = 0
                    stepsv = 0
                    valid_summary = []
                    for Xv, Yv, Zv in gen_batch(valid_data_x, 
                                        valid_data_y, num_steps, batch_size, w_slice):
                        valid_loss_, valid_pred = sess.run([g['total_loss'], g['prediction']], 
                                                         feed_dict = {g['x']: Xv, g['y']:Yv})
                        valid_loss += valid_loss_
                        valid_summary.append(np.array([valid_pred, Yv, Zv]))
                        valid_r += np.corrcoef(valid_pred, Yv)[0,1]
                        stepsv += 1
                    valid_summary = np.concatenate(valid_summary, axis=1)
                    summary = validsum(valid_summary.T)
                    print('Average validation loss for Epoch', idx, ":", valid_loss/stepsv)
                    print('Average validation R for Epoch', idx, ":", valid_r/stepsv)
                    print('validation R for Epoch', idx, ":", np.corrcoef(summary[:,0], summary[:,1])[0,1])
            training_loss += training_loss_
        print('Average training loss for Epoch', idx, ":", training_loss/steps)
    if isinstance(save, str):
        saver = tf.train.Saver()
        saver.save(sess, save)
    return training_loss, valid_summary

There will be a predicted value for each time window. Average the predicted values of time windows corresponding to individuals and returen the summary matrix with prediction, response and individual label.

In [ ]:
def validsum(valid_summary):
    Z = valid_summary[:,2]
    Y = valid_summary[:,1]
    preds = valid_summary[:,0]
    zs = np.unique(Z)
    results=[]
    for i in np.arange(len(zs)):
        z = zs[i]
        results.append([np.mean(preds[Z == z]), np.mean(Y[Z == z]), z])
    return np.array(results)

Get the device information.

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

device_lib.list_local_devices()

Time test for training 10 epochs using 4 GPUs.

In [22]:
num_steps = 100
batch_size = 32
state_size = 256
num_layers = 2
learning_rate = [1e-5, 1e-5]
input_x_dim = 236
w_slice = 40
workdir = '/Project/models'
os.chdir(workdir)
t = time.time()
g = build_graph(state_size = state_size,batch_size = batch_size, 
                num_steps = num_steps, num_layers = num_layers,
                learning_rate = [1e-4, 1e-4], input_x_dim = input_x_dim, reg_scale = [1e-5, 1e-5], keep_prob = [0.4, 0.4], hidden_states=32, devices=['/gpu:0', '/gpu:1', '/gpu:2', '/gpu:3'])
print('It took', time.time() - t, 'seconds to build the graph.')
t = time.time()
raw_data_x = np.array(adhd_roimean)
raw_data_y = adhd_response
valid_data_x = np.array(gradCPT_roimean)
valid_data_y = gradCPT_response
_, valid_summary = train_network(g, 10, num_steps, batch_size, raw_data_x = raw_data_x, raw_data_y = raw_data_y, 
              valid_data_x = valid_data_x, valid_data_y = valid_data_y, w_slice=w_slice, save = workdir + '/model.ckpt')
print("It took", time.time() - t, "seconds to train for 10 epochs.")


It took 3.484865665435791 seconds to build the graph.
Average validation loss for Epoch 0 : 1.61791519324
Average validation R for Epoch 0 : 0.106097355416
validation R for Epoch 0 : 0.286413504124
Average training loss for Epoch 0 : 1.17190149968
Average validation loss for Epoch 1 : 1.22665141026
Average validation R for Epoch 1 : -0.00491916869397
validation R for Epoch 1 : -0.0725694246051
Average training loss for Epoch 1 : 1.14686973479
Average validation loss for Epoch 2 : 1.20344215631
Average validation R for Epoch 2 : 0.0292518055246
validation R for Epoch 2 : 0.107812127604
Average training loss for Epoch 2 : 1.14369151156
Average validation loss for Epoch 3 : 1.23790300886
Average validation R for Epoch 3 : -0.0604159976538
validation R for Epoch 3 : -0.220298838491
Average training loss for Epoch 3 : 1.14475248939
Average validation loss for Epoch 4 : 1.23049556216
Average validation R for Epoch 4 : 0.00635642074072
validation R for Epoch 4 : -0.0790651404598
Average train

In [39]:
num_steps = 60
batch_size = 16
state_size = 256
num_layers = 2
learning_rate = [1e-5, 1e-5]
input_x_dim = 236
w_slice = 30
workdir = '/Project/models'
os.chdir(workdir)
t = time.time()
g = build_graph(state_size = state_size,batch_size = batch_size, 
                num_steps = num_steps, num_layers = num_layers,
                learning_rate = [1e-5, 1e-5], input_x_dim = input_x_dim, reg_scale = [0.00001, 0.00001], keep_prob = [0.4, 0.4], hidden_states=256, devices=['/gpu:0', '/gpu:1', '/gpu:2', '/gpu:3'])
print('It took', time.time() - t, 'seconds to build the graph.')
t = time.time()
saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess, workdir + '/model.ckpt')
_, valid_summary = train_network(g, 10, num_steps, batch_size, raw_data_x = raw_data_x, raw_data_y = raw_data_y, 
              valid_data_x = valid_data_x, valid_data_y = valid_data_y, w_slice=w_slice, save = workdir + '/model1.ckpt', sess = sess)
print("It took", time.time() - t, "seconds to train for 10 epochs.")

It took 3.6968748569488525 seconds to build the graph.
INFO:tensorflow:Restoring parameters from /home/fas/zhao/dj333/Project/Imaging/share/lstm_prediction/LSTM1/tower/models/model.ckpt
Average validation loss for Epoch 0 : 36.0225739913
Average validation R for Epoch 0 : -0.028794941792
Average training loss for Epoch 0 : 4.59227508617
Average validation loss for Epoch 1 : 36.1441309958
Average validation R for Epoch 1 : 0.0255554786764


KeyboardInterrupt: 